In [11]:
# TODO: Add memory function
# TODO: Add assertion / feedback loop
import os
os.chdir('../..')
import dspy
import json
from pathlib import Path
from dspy.datasets import DataLoader
import sys
notebook_path = Path(os.getcwd())
sys.path.insert(0, str(notebook_path.parent.parent))
from llm_api import LLM_API
template1 = """
{text}

As a professional narrative writing expert, you know:

1. Types of narrative:
   - Diary
   - Epistolary narrative 
   - Prose
   - Event/Character/Scene narrative
   - Novel (character, plot, setting)
   - Dialogue (like drama)

2. Types of characters:
   - Fictional first-person "I"
   - First-person as the author themselves
   - Fictional people
   - Real people
   - Anthropomorphized animals/objects/concepts

Your role is to guide another AI in transforming any given text into a narrative form. You won't be writing the narrative itself, but rather completing the following three tasks, which another AI will reference to generate the narrative:
1. Summarize the content of the given text.
2. Choose the type of narrative, person(First person, second person, third person), and type of character, and provide a theme and background information according to the given text.
3. Based on the given text, analyze how to transform the text into the (type_of_narrative, person, type_of_character, theme).

Output in a code block enclosed by triple backticks in the following format:

```
1. {{summary}}
2. Type of narrative: {{type_of_narrative}}; Person: {{person}};  Type of character: {{type_of_character}}; Theme: {{theme}}; Background information: {{background_information}}
3. Analysis: {{analysis}}
```

Do not provide any other information except the guidance of the three questions above.

---
Memory area:
Here is the number of times you have chosen specific type_of_narrative and type_of_character:

1. Types of narrative:
   - Diary: {diary_cnt}
   - Epistolary narrative: {epistolary_narrative_cnt}
   - Prose: {prose_cnt}
   - Event/Character/Scene narrative: {event_character_scene_narrative_cnt}
   - Novel (character, plot, setting): {novel_cnt}
   - Dialogue (like drama): {dialogue_cnt}

2. Types of characters:
   - Fictional first-person "I": {fictional_first_person_cnt}
   - First-person as the author themselves: {first_person_as_author_cnt}
   - Fictional people: {fictional_people_cnt}
   - Real people: {real_people_cnt}
   - Anthropomorphized animals/objects/concepts: {anthropomorphized_cnt}

你需要多样化 type_of_narrative 和 type_of_character 的选择，使得分布平均，以便生成多样化的记叙文。
"""


template2 ="""
{text}

As a professional narrative writing expert, your task is to transform any given text, which could be of any type, into type_of_narrative. The requirements are:

1. Avoid unnecessary filler content.
2. Integrate sufficient background information about the given text in the narrative.
3. Refer to the following suggestions to complete this task:

在写记叙文时，注意因果逻辑
{suggestions}
"""

In [12]:
dl = DataLoader()

dataset = dl.from_csv("data/wikibooks_splited.csv", fields=("text",), input_keys=("text",))

splits = dl.train_test_split(dataset, train_size=0.8) 
trainset = splits['train']
devset = splits['test']

print(len(trainset), len(devset))
print(trainset[0], devset[0], sep="\n")

1538 385
Example({'text': '[\'Ict@innovation: Free your IT Business in Africa/5-6\\nModule 5.6 Innovation in FOSS Business[edit | edit source]\\nDuration[edit | edit source]\\n1:00hrs\\nDelivery method[edit | edit source]\\nFor instructional purpose, it is advised that trainers/lectures use lectures, role play and group and individual exercises as a major means of delivering this module.\\nIntroduction[edit | edit source]\\nThe debate on how FOSS spins ICT and business innovation is well documented. Special reference is made to the innovative capacity of FOSS (Sanjiva and Jivaka, 2003) and factors which hinder and support technology innovation in developing countries (Jean-Eric Aubert, 2004). This module addresses the structural nature of FOSS, that is, commoditisation which increases the innovative base of developing, customizing, and marketing FOSS products. Strategies for innovation in a FOSS context, the concept of open innovation, and where innovation happens in FOSS are the sub-m

In [15]:
output_list = []  
llm = LLM_API()

for data in dataset[:2]:
    prompt1 = template1.format(text=data['text'])
    response = llm.generate_text(prompt1, model = "deepseek")
    
    prompt2 = template2.format(text=data['text'], suggestions=response)
    
    response = llm.generate_text(prompt2, model = "deepseek")
    
    output_list.append({"original_text": data['text'], "transformed_text": response, "type" : "narration", "tag" : []})

with open("result/genre_transformation/narration.json", "w") as f:
    f.write(json.dumps(output_list, indent=4))